In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import json
import pandas as pd
import os

/home/csmajs/jxie031/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/csmajs/jxie031/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/csmajs/jxie031/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/csmajs/jxie031/.local/lib/python3.6/site-packages/tensorflow/

In [2]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [3]:
# load all resource files
path = 'cleanedDCinboxResource/'
dest_path = 'encodedDCinboxResource/'
dirs = os.listdir(path)
dirs

['healthcare.json.csv',
 'pet.json.csv',
 'business.json.csv',
 'fashion beauty.json.csv',
 'fitness sports.json.csv',
 'food beverage.json.csv',
 'media enterainment.json.csv',
 'lifecycle home.json.csv',
 'consumer.json.csv',
 'green cleantech.json.csv',
 'children education.json.csv',
 'software tech.json.csv']

In [4]:
# load the csv data into the dataframe
df_dict = {}
for file in dirs:
    df_dict[file] = pd.read_csv(path + file)
    
# test if the data be read or not
df_dict['healthcare.json.csv'].head()

,sentences
0,Congressman Carol Miller ?
1,"Dear Friend, Happy Presidents' Day weekend!"
2,On this day we honor all that our Presidents h...
3,This week was filled with many exciting develo...
4,I introduced legislation to fill the remaining...


In [5]:
print(df_dict['healthcare.json.csv']['sentences'].to_numpy())

['Congressman Carol Miller ?'
 "Dear Friend, Happy Presidents' Day weekend!"
 'On this day we honor all that our Presidents have done to lead our nation into the prosperous and powerful one it is today.'
 ...
 'There would never be enough time in the world to fully encapsulate what he meant to me and the people of Michigans Thirteenth Congressional District, but please watch my full address in honor of his tremendous legacy and know that it is that tremendous legacy that fuels my work on your behalf every single day.'
 'My sincere condolences go out to his family, friends, and loved ones.'
 'May Congressman Conyers rest in peace as we continue his fight for jobs, justice, and peace.']


In [6]:
for (key, data) in df_dict.items():
    # data['embeded'] = embed(data['sentences'].to_numpy()).numpy()
    data['embed'] = embed(data['sentences'].to_numpy()).numpy().tolist()
    

In [7]:
df_dict['healthcare.json.csv'].head()

,sentences,embed
0,Congressman Carol Miller ?,"[0.05492091551423073, 0.023301545530557632, 0...."
1,"Dear Friend, Happy Presidents' Day weekend!","[-0.03798495605587959, -0.06996205449104309, 0..."
2,On this day we honor all that our Presidents h...,"[-0.06237169727683067, -0.08522521704435349, 0..."
3,This week was filled with many exciting develo...,"[-0.04813029244542122, -0.04647025093436241, -..."
4,I introduced legislation to fill the remaining...,"[-0.04485123232007027, -0.07864920049905777, -..."


In [9]:
for (key, data) in df_dict.items():
    data.to_csv(dest_path + key , index=False)